# 0.1. imports

In [1]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

# 0.2. config

In [2]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

# 0.3. load `pylcaio`

In [3]:
sys.path.append(str(Path(Path.home(), config['path_dir_repo_pylcaio'])))
import pylcaio

# 0.4. file paths
## 0.4.1. names

In [4]:
%%capture
print(ecoinvent_version_name := config['ecoinvent_version_name'])

## 0.4.2. directories

In [5]:
%%capture
print(path_dir_data := Path(Path.home(), config['path_dir_data']))
print(path_dir_data_raw := Path(path_dir_data, config['path_dir_data_raw']))
print(path_dir_data_processed := Path(path_dir_data, config['path_dir_data_processed']))

## 0.4.3. files

In [6]:
%%capture
print(path_dir_exiobase_2011_pxp := Path(path_dir_data_raw, 'IOT_2011_pxp.zip'))
print(path_file_e2m_pickle := Path(path_dir_data_processed, (config['ecoinvent_version_name'] + config['e2m_pickle_filename'])))

In [7]:
%%capture
print(path_pylcaio_database_loader_class_instance := Path(path_dir_data_processed, config['pylcaio_database_loader_class_instance']))
print(path_pylcaio_class_instance_before_hybrid := Path(path_dir_data_processed, config['pylcaio_class_instance_before_hybrid']))
print(path_pylcaio_class_instance_after_hybrid := Path(path_dir_data_processed, config['pylcaio_class_instance_after_hybrid']))

# 1. load databases
## 1.1. load `Exiobase`

In [8]:
exiobase: pymrio.IOSystem = pymrio.parse_exiobase3(path_dir_exiobase_2011_pxp)

## 1.2. load `Ecoinvent`

In [9]:
with gzip.open(path_file_e2m_pickle, 'rb') as filestream:
    ecoinvent: dict = pd.read_pickle(filestream)

# 2. main `pylcaio` functionality

In [11]:
database_loader: pylcaio.DatabaseLoader  = pylcaio.DatabaseLoader(
    lca_database_processed = ecoinvent,
    io_database_processed = exiobase,
    lca_database_name_and_version = 'ecoinvent3.5',
    io_database_name_and_version = 'exiobase3'
)

In [12]:
with open(path_pylcaio_database_loader_class_instance, 'wb') as file_handle:
    pickle.dump(obj = database_loader, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
lcaio_object: pylcaio.LCAIO = database_loader.combine_ecoinvent_exiobase(
    complete_extensions = False,
    impact_world = False,
    regionalized = False)

No path for the capital folder was provided. Capitals will not be endogenized


In [14]:
with open(path_pylcaio_class_instance_before_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
lcaio_object.hybridize(
    price_neutral_cut_off_matrix = 'STAM',
    capitals = False,
    priceless_scaling = True)

Indentifying Rest of World regions...
Updating electricity prices...
Calculating productions volumes...
Adjusting low production volume processes...
Extending inventory...
Building H matrix...


/Users/michaelweinold/github/pylcaio/src/pylcaio.py:874: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.H = self.H.append([self.H] * (self.number_of_countries_IO + self.number_of_RoW_IO - 1))


Building geography concordance...
Filter H matrix...
Build Cut-off matrix...
Add processes with 'priceless scaling' to Cut-off matrix...


In [ ]:
with open(path_pylcaio_class_instance_after_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)